### Import defs from preprocessing:

In [3]:
from ipynb.fs.full.preprocessing import BacteriaDataset, train_data

## Network definiton

-  Image size remains the same through convolution layers, but channels are affected, e.g: `[batch_size,128,32,32] –> [batch_size,256,32,32]`
- Pooling layers reduce image dimensions e.g: `[batch_size,256,32,32] –> [batch_size,256,16,16]`

In [28]:
from re import X
import torch.nn as nn
import torch.nn.functional as F
import torch

class ConvNetwork(nn.Module):
    '''
    Convolutional network definition, parameters output dimensions and hyperparameters
    '''
    def __init__(self, input_channels:int,conv_out1:int,conv_out2:int,kernel1:int,kernel2:int,fc1:int,fc2:int,fco:int,stride:int = 1,padding:int = 0,):
        super().__init__()

        self.conv1 = nn.Conv2d(input_channels,conv_out1,kernel_size=kernel1,stride=stride,padding=padding)
        self.bn1 = nn.BatchNorm2d(conv_out1,)
        self.conv2 = nn.Conv2d(conv_out1,conv_out2,kernel_size=kernel2,stride=stride,padding=padding)
        self.bn2 = nn.BatchNorm2d(conv_out2) 
        if (conv_out2 % 2 != 0):
            raise ValueError("'conv_out2' must be divisble by 2, for pooling layer")
            
        pool1_dim = int(conv_out2/2)
        

        self.pool1 = nn.AdaptiveAvgPool2d(pool1_dim) # halves number of channels
        self.fc1 = nn.Linear(pool1_dim*(conv_out2**2),fc1) 
        self.fc2 = nn.Linear(fc1,fc2)
        self.fco = nn.Linear(fc2,fco) # 24 categories

    def forward(self, x:torch.Tensor):
        device = x.device # to transfer x to cpu 

        x_conv1 = self.conv1(x)
        x_bn1 = self.bn1(x_conv1)
        x_relu1 = F.relu(x_bn1)
        
        x_conv2 = self.conv2(x_relu1)
        x_bn2 = self.bn2(x_conv2)
        x_relu2 = F.relu(x_bn2)

        #weird mps support on adaptive poolin > transfer 'x' to cpu:
        if x.device == 'mps':
            x_relu2.to('cpu')
        pool1 = self.pool1(x_relu2)
        pool1.to(device)

        x_relu3 = F.relu(pool1)
        x_fc1 = self.fc1(x_relu3)
        x_relu4 = F.relu(x_fc1)
        x_fc2 = self.fc2(x_relu4)
        x_relu5 = F.relu(x_fc2)
        x_fco = self.fco(x_relu5)

        return x_fco


### Test `ConvNetwork()`

In [29]:
import torchvision.transforms.v2 as v2
from torch.utils.data import DataLoader
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)
print(f'Device: {device}')

pixel_means = [0.0550, 0.0316, 0.0273]
pixel_stds = [0.0206, 0.0259, 0.0257]
image_size = 256
batch_size = 10

transform = v2.Compose([
    v2.Resize([image_size,image_size]),
    v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
    v2.Normalize(pixel_means,pixel_stds),
    ])

nw_test_dataset = train_dataset = BacteriaDataset(annotations=train_data['encoded_cats'],images=train_data['image_path'],transform=transform,device=device)
nw_test_item = nw_test_dataset.__getitem__(0)
nw_test_dataloader = DataLoader(dataset=nw_test_dataset,batch_size = batch_size,shuffle=True,)

test_network = ConvNetwork(
    input_channels=3,
    conv_out1=32,
    conv_out2=128,
    kernel1=10,
    kernel2=10,
    fc1=64,
    fc2=32,
    fco=24
)
test_network.to(device=device)
nw_test_batch,_ = next(iter(nw_test_dataloader))
test_output = test_network(nw_test_batch)
print('done')

Device: mps


RuntimeError: MPS backend out of memory (MPS allocated: 8.93 GB, other allocations: 94.70 MB, max allowed: 9.07 GB). Tried to allocate 74.47 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).